In [78]:
import os
import rasterio as rio
import numpy as np
from rasterio.windows import Window
from glob import glob

# Mount Inference Data

In [37]:
# !sudo mount -t nfs4 -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2,noresvport 172.31.91.151:/ ./efs_inference_data

# Windowing

In [38]:
def get_windows(img_dim, patch_size=(240, 240), stride=(240, 240)):
    patch_size = np.array(patch_size)
    stride = np.array(stride)
    img_dim = np.array(img_dim)
    
    max_dim = (img_dim//patch_size)*patch_size - patch_size

    ys = np.arange(0, img_dim[0], stride[0])
    xs = np.arange(0, img_dim[1], stride[1])

    tlc = np.array(np.meshgrid(ys, xs)).T.reshape(-1, 2)
    tlc = tlc[tlc[:, 0] <= max_dim[0]]
    tlc = tlc[tlc[:, 1] <= max_dim[1]]
    
    windows = []
    for y,x in tlc.astype(int):
        windows.append(Window(x, y, patch_size[1], patch_size[0]))

    return windows

In [82]:
def add_ndvi(data, dtype_1=rio.float32):
    
    nir = data[7].astype(dtype_1)
    red = data[3].astype(dtype_1)

    # Allow division by zero
    np.seterr(divide='ignore', invalid='ignore')

    # Calculate NDVI
    ndvi = np.zeros(nir.shape, dtype=dtype_1)
    ndvi = ((nir - red) / (nir + red)).astype(dtype_1)

    # Rescaling for use in 16bit output

    ndvi = (ndvi + 1) * (2**15 - 1)

    # Add NDVI band to end of array    
    rast = np.concatenate((data,[ndvi]),axis=0)
    
    return rast


In [83]:
def output_windows(granule_dir,out_path,patch_size=100,
                   stride=100,SAVE=False,SAVE_INDIVIDUAL=False,
                   bands=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]):
    
    granule_list = glob(f'{granule_dir}/*.tif')
    
    for granule_path in granule_list[0:1]:

        raster = rio.open(granule_path)

        windows = get_windows(raster.shape, (patch_size, patch_size), (stride, stride))

        for j, window in enumerate(windows):

            data = raster.read((bands),window=window, masked=True)
            
            data = add_ndvi(data)
            
            return data
            

#             print(granule_path, j, data)

#             print(j, window, data.shape)

        #     if np.any(data.mask):
                # THE PATCH CONTAINS NODATA PIXELS, HANDLE AS NEEDED

        #     data = DO_PREPROCESSING(data.data)
        #     result = DO_INFERENCE(data)

#             if SAVE:

#                 if SAVE_INDIVIDUAL:
#                     meta = raster.meta.copy()
#                     # Get the window specific transform - IF we want to save windows independantly
#             #         trans = raster.window_transform(window)
#                     meta.update({
#             #                 'transform': trans,
#                             'dtype': src.dtype
#                         })

#                     with rasterio.open(f"{out_path}/some_chip_{j}.tif", 'w', **meta) as dest:
#                         dest.write(data)
#                 else:
#                     meta = raster.meta.copy()

#                     with rasterio.open(f"{out_path}/some_chip_{j}.tif", 'w+', **meta) as dest:
#                         dest.write(data, window=window)

In [84]:
granule_dir = "./efs_inference_data/"
out_path = "./windows/"

data = output_windows(granule_dir,out_path)